In [ ]:
from fastai import *
from fastai.tabular import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_x=pd.read_csv('../input/lish-moa/train_features.csv')
train_y=pd.read_csv('../input/lish-moa/train_targets_scored.csv')

test_x=pd.read_csv('../input/lish-moa/test_features.csv')

In [ ]:
print(train_x.shape)
train_x.head()

In [ ]:
print(train_y.shape)
train_y.head()

In [ ]:
for col in train_y.columns:
  if col!='sig_id':
    print(train_y[col].value_counts())

In [ ]:
print(test_x.shape)
test_x.head()

In [ ]:
train_x.info()

There are
>872 float dtype 

> 1 int64  (cp_time )

> 3 object (sig_id , cp_time , cp_dose)

In [ ]:
train_x.iloc[:,:10].info()

In [ ]:
test_x.info()

In [ ]:
train_y.info()

In [ ]:
print('missing values in train_x:',sum(train_x.isnull().sum()))
print('missing values in test_x:',sum(test_x.isnull().sum()))
print('missing values in train_y:',sum(train_y.isnull().sum()))

In [ ]:
print('train:')
print(train_x.cp_time.value_counts())
print(' ')
print(train_x.cp_type.value_counts())
print(' ')
print(train_x.cp_dose.value_counts())
print('--------------------------------------')
print('test:')
print(test_x.cp_time.value_counts())
print(' ')
print(test_x.cp_type.value_counts())
print(' ')
print(test_x.cp_dose.value_counts())

In [ ]:
cat=['cp_time','cp_dose','cp_type']
cont=[x for x in train_x.columns if x not in cat and x!='sig_id']
depv=[x for x in train_y.columns if x!='sig_id']
print('categorical features:',len(cat))
print('continuous features:',len(cont))
print('dependent variables:',len(depv))

In [ ]:
train=train_x
print('train_shape:',train.shape)

In [ ]:
print(len(cat)+len(cont)+len(depv)+1)
train[depv]=train_y[depv]
print('train_shape:',train.shape)

In [ ]:
train.head()

## **Model**

In [ ]:
procs = [FillMissing,Categorify,Normalize]
import torch.nn as nn

In [ ]:
procs = [FillMissing,Categorify,Normalize]
data=(TabularList.from_df(train, cont_names=cont, cat_names=cat, procs=procs)
                .split_by_rand_pct(valid_pct=0.15,seed=42)
                .label_from_df(cols=depv)
                .add_test(TabularList.from_df(test_x,procs=procs, cont_names=cont, cat_names=cat))
                .databunch())

In [ ]:
learn=tabular_learner(data,layers=[1000,1000,1000])
learn.loss_func = nn.BCEWithLogitsLoss()

In [ ]:
#learn

In [ ]:
learn.fit_one_cycle(7, max_lr=slice(1e-4),wd=0.1)

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(7, max_lr=slice(1e-5),wd=0.05)

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(7, max_lr=slice(5e-6),wd=0.02)

In [ ]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
v=math.pow(10,-15)
def const1(a):
  for i in range(a.shape[0]):
    for j in range(a.shape[1]):
      if a[i][j]<v:
        a[i][j]=v
      elif a[i][j]>1-v:
        a[i][j]=1-v
  return a

In [ ]:
const1(preds)

In [ ]:
sub=pd.read_csv('../input/lish-moa/sample_submission.csv')
sub.head()

In [ ]:
sub[depv]=preds
print(sub.shape)
sub.head()

In [ ]:
sub.to_csv('submission.csv', header=True,index=False)